# Laboratorio 2
## Microestructuras y Sistemas de Trading
### Valentina Escobar y Pablo Mendoza

El presente laboratorio tiene como objetivo analizar el comportamiento de un ETF de acuerdo a la conformación de este y al movimiento de los precios que se presentaron entre el 31 de marzo del 2017 hasta el 29 de marzo del 2018 con el fin de identificar que tan conveniente es para un inversionista rebalancear diariamente durante un año comprando $n$ acciones cuando el rendimiento diario sea de -1%. Se contrastará el capital inicial con el capital final deshaciendo todas las pocisiones del portafolo el 1 de Marzo de 2018. Se realizan diferentes medidas de desempeño que permitan comparar los portafolios resultantes.

## Librerías y Funciones 

In [1]:
# Librerias
import quandl
import datetime
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
quandl.ApiConfig.api_key = "My9ygJsz7yTDx_9jcneH" # Quandl API key
fechas = list(['2017-03-31','2018-03-29'])

In [89]:
data_csv = pd.read_csv('/Users/pablomendoza/Documents/ITESO/Trading/Lab_1/Trading/Data/IJH_holdings-1.csv', skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'])
tick = list(data_csv.index)
weights = data_csv['Weight (%)']
#del data_csv
def Download_QuandL(fechas:'Dates_list ', tick:'Tickers of Stock'):
    dates = { 'gte': fechas[0], 'lte': fechas[1] }
    column = { 'columns': ['date', 'Open', 'Close']}
    data = quandl.get_table('WIKI/PRICES', qopts = column, ticker = tick, date = dates, paginate=True)# ticker = 'WIKI/AAPL' #'AAPL.US'
    data = data.iloc[::-1]
    data = data.reset_index()
    data = data.drop(['None'],1)
    return data
def All_Data(fechas:'Dates_list ', tick:'Ticker'):
    d = {} # Crea un Diccionario Vacío
    for i in range(len(tick)): # ciclo que descarga los datos de Quandl y los guarda como DataFrames en un Diccionario
        nom = str(tick[i])
        d["_{0}".format(nom)] = Download_QuandL(fechas, tick[i])
    return d
def Rebalanceo(df, Com, Cap_I, R_R, n_a, we, op):
    """
    Esta funcion hace los rebalanceos de cada activo de forma diaria por una año
    :param df: DataFrame del Activo
    :param Com: Comisión
    :param Cap_I: Capital Inicial
    :param R_R: Rendimiento de Rebalanceo
    :param n_a: Acciones que se compran al rebalancear
    :param we: Weight of each stock on the porfolio
    :param op: Dataframe to save operation
    :return: 
    """
    a = (df['close']/df['open'])-1
    P_se = list(df['open']*.99)
    add_s = list()
    Opp = list()
    comi = list()
    In_shares = np.floor((we*Cap_I)/df['close'][0])
    for i in range(len(df)):
        if a.iloc[i] < R_R:
            add_s.append(n_a)
            Opp.append('Compra')
            comi.append(Com*df['close'].iloc[i]*n_a)

        else:
            add_s.append(0)
            Opp.append('-')
            comi.append(0)
    df = df.join(pd.DataFrame(add_s, columns = ['# Added Shares']))
    df = df.join(pd.DataFrame(Opp, columns = ['Operacion']))
    df = df.join(pd.DataFrame(comi, columns = ['Comisión']))
    df = df.join(pd.DataFrame(P_se, columns = ['Precio de Señal']))
    Tot_Acciones = pd.DataFrame(list(df['# Added Shares'].cumsum()), columns = ['Total Shares'])+In_shares
    df = df.join(Tot_Acciones)
    Val_to_port = pd.DataFrame(list(df['close']*df['Total Shares']), columns = ['Value to ETF'])
    df = df.join(Val_to_port)
    
    frames = [op, df[df.Operacion == 'Compra']]
    op = pd.concat(frames)

    
    return df, op
def Clean_Dicc(dic:'Diccionario a limpiar',sizes:'Tamaño deseado del DF'):
    y = list(dic.keys())
    for h in y:
        if len(dic[h])!=248:
            del dic[h]
    return dic

In [4]:
Data = All_Data(fechas, tick)

In [5]:
d_full = Clean_Dicc(Data,248)

In [102]:
tick = list(d_full.keys())
d2 = {}
op = pd.DataFrame(columns=['date', 'open', 'close', '# Added Shares', 'Operacion', 'Comisión', 'Precio de Señal', 'Total Shares', 'Value to ETF'])
for j in range(len(tick)):
    tabla , op = Rebalanceo(d_full[tick[j]],0.0003,1000000,-0.01,10,weights[j], op)
    d2["Data{0}".format(j)] = tabla
op_2 = op.groupby('date')
# for key, item in op_2:
#     print(op_2.get_group(key), "\n\n")

In [103]:
d2['Data0']['Value to ETF']

0      609945.75
1      604945.50
2      598616.75
3      588363.70
4      586579.75
5      586409.85
6      585985.10
7      589807.85
8      581142.95
9      578679.40
10     588278.75
11     583181.75
12     587004.50
13     591421.90
14     593290.80
15     600936.30
16     602635.30
17     609516.25
18     618860.75
19     577575.05
20     585475.40
21     592866.05
22     585730.25
23     586155.00
24     589468.05
25     582332.25
26     573747.30
27     580041.00
28     577744.65
29     581401.80
         ...    
218    796681.60
219    813656.35
220    819923.95
221    816616.05
222    808085.15
223    811306.00
224    816354.90
225    830631.10
226    839162.00
227    832282.50
228    830278.05
229    818861.40
230    827227.80
231    831759.60
232    838208.70
233    842304.75
234    850671.15
235    868711.20
236    867491.10
237    861826.35
238    859211.85
239    855028.65
240    855725.85
241    851891.25
242    855638.70
243    856945.95
244    840823.20
245    832021.